In [1]:
import json
import pickle

import numpy as np
import pandas as pd
import os
from sklearn import preprocessing

from datasets import DATASET

Dataset(name='codec', root=WindowsPath('d:/IRProj/bug-localization/master/../data/CODEC'), src=WindowsPath('d:/IRProj/bug-localization/master/../data/CODEC/gitrepo'), bug_repo=WindowsPath('d:/IRProj/bug-localization/master/../data/CODEC/bugrepo/repository.xml'))


In [2]:
# checking the matching tokens between the bug reports and source files
def check_matchings(src_files, bug_reports):
    

    scores = []
    for report in bug_reports.values():
        matched_count = []
        summary_set = report.summary
        pos_tagged_sum_desc = (report.pos_tagged_summary['stemmed'] + report.pos_tagged_description['stemmed'])

        for src in src_files.values():
            if src.file_name['stemmed']:
                common_tokens = len(set(summary_set['stemmed']) & set([src.file_name['stemmed'][0]]))
            else:
                common_tokens = 0

            matched_count.append(common_tokens)

        # Here no files matched a summary
        if sum(matched_count) == 0:
            matched_count = []
            for src in src_files.values():
                common_tokens = len(set(pos_tagged_sum_desc) & set(src.file_name['stemmed'] + src.class_names['stemmed'] + src.method_names['stemmed']))

                if not common_tokens:
                    common_tokens = (len(set(pos_tagged_sum_desc) & set(src.comments['stemmed'])) - len(set(src.comments['stemmed'])))

                if not common_tokens:
                    common_tokens = (len(set(pos_tagged_sum_desc) & set(src.attributes['stemmed'])) - len(set(src.attributes['stemmed'])))

                matched_count.append(common_tokens)

        min_max_scaler = preprocessing.MinMaxScaler()

        intersect_count = []
        for count in matched_count:
            intersect_count.append(float(count))

        intersect_count = np.array(intersect_count).reshape(-1, 1)

        normalized_count = np.concatenate(min_max_scaler.fit_transform(intersect_count))

        scores.append(normalized_count.tolist())

    return scores


In [3]:
listOfList = []

In [4]:
def main():

    # Unpickle preprocessed data
    with open(DATASET.root / 'preprocessed_src.pickle', 'rb') as file:
        src_files = pickle.load(file)
    with open(DATASET.root / 'preprocessed_reports.pickle', 'rb') as file:
        bug_reports = pickle.load(file)

    scores = check_matchings(src_files, bug_reports)
    for i in scores:
        listOfList.append(i)
    # print(len(scores[1]))

    # Saving scores in a json file
    with open(DATASET.root / 'token_matching.json', 'w') as file:
        json.dump(scores, file)


In [5]:
main()

In [6]:
path = os.getcwd()
path+="\\table.csv"
# path

In [7]:
mrr = []
mean_avgp = []
df = pd.read_csv(path)

In [8]:
for the in listOfList:
    another = []
    for i,j in enumerate(the):
        another.append((i+1,j))
    # print(another)
    another = sorted(another, key = lambda x: x[1],reverse=True)
    # print(another)
    relevent_ranks = []
    for rank,score in another:
        if score!=0:
            relevent_ranks.append(rank)
    # relevent_ranks = sorted(relevent_ranks)
    # print(relevent_ranks)
    if len(relevent_ranks)==0:
        mean_avgp.append(0)
    else:
        someAnother = sorted(relevent_ranks)
        # print(someAnother)
        for j, rank in enumerate(someAnother):
            # print(j,rank)
            l = len(someAnother[:j + 1])
            if rank!=0:
                t = l / rank
                mean_avgp.append(np.mean(t))
            else:
                mean_avgp.append(0)
    # print(relevent_ranks)
    if len(relevent_ranks):
        position = (relevent_ranks[0])
        if position:
            mrr.append(1/position)
        else:
            mrr.append(0)
    else:
        mrr.append(0)
# print(mrr)
# print(np.mean(mrr))

# print(np.mean(mean_avgp))

some = ['Token_matching_score',DATASET.name,np.mean(mrr),np.mean(mean_avgp)]
df.loc[len(df)] = some

In [ ]:
# df.drop_duplicates(subset = "Dataset" ,keep = "first", inplace = True)
# df.to_csv(path,index=None)
# df

In [10]:
print("Top 10 retrieved files:")
another[0:10]

top 10 retrieved files:


[(19, 1.0),
 (20, 1.0),
 (49, 1.0),
 (64, 1.0),
 (69, 1.0),
 (71, 1.0),
 (75, 1.0),
 (13, 0.9968454258675078),
 (16, 0.9968454258675078),
 (36, 0.9968454258675078)]